receiver position (x, y, z)
and time correction d for simultaneous satellite positions (Ai
, Bi
, Ci) are [-4.17727096e+01 -1.67891941e+01  6.37005956e+03] and -3.20156583e-03 respectively

In [1]:
# problem 1

import numpy as np


def receiver_position(x, y, z, d):
    # Sort the initial position
    u = np.array([x, y, z, d])

    # Set number of iterations
    n = 10

    # Initial positions and time of satellites that are given
    A1, B1, C1 = 15600, 7540, 20140
    A2, B2, C2 = 18760, 2750, 18610
    A3, B3, C3 = 17610, 14630, 13480
    A4, B4, C4 = 19170, 610, 18390

    t = np.array([0.07074, 0.07220, 0.07690, 0.07242])

    # Speed of light
    c = 299792.458

    x0, y0, z0, d0 = x, y, z, d

    for i in range(n):
        # Find F
        F = np.array([(x0 - A1)**2 + (y0 - B1)**2 + (z0 - C1)**2 - (c*(t[0] - d0))**2,
                      (x0 - A2)**2 + (y0 - B2)**2 + (z0 - C2)**2 - (c*(t[1] - d0))**2,
                      (x0 - A3)**2 + (y0 - B3)**2 + (z0 - C3)**2 - (c*(t[2] - d0))**2,
                      (x0 - A4)**2 + (y0 - B4)**2 + (z0 - C4)**2 - (c*(t[3] - d0))**2])

        # Find dF
        dF = np.array([[2*(x0 - A1), 2*(y0 - B1), 2*(z0 - C1), 2*(c**2*(t[0] - d0))],
                       [2*(x0 - A2), 2*(y0 - B2), 2*(z0 - C2), 2*(c**2*(t[1] - d0))],
                       [2*(x0 - A3), 2*(y0 - B3), 2*(z0 - C3), 2*(c**2*(t[2] - d0))],
                       [2*(x0 - A4), 2*(y0 - B4), 2*(z0 - C4), 2*(c**2*(t[3] - d0))]])

        # Apply Newton Method to find next position


        v = np.linalg.solve(dF, -F)

        u += v

        x0, y0, z0, d0 = u[0], u[1], u[2], u[3]

    return u

p = receiver_position(0.0,0.0,6370.0,0.0)
print(p)


[-4.17727096e+01 -1.67891941e+01  6.37005956e+03 -3.20156583e-03]


In [21]:
# Problem 2

import numpy as np

def getSphericalCoordinates(phi, theta, ρ=26570):
    x = ρ * np.cos(phi) * np.cos(theta)
    y = ρ * ρ * np.cos(phi) * np.sin(theta)
    z = ρ * np.sin(phi)
    return np.array([x, y, z])

def calculateRange(satellitePosition):
    x_diff = satellitePosition[0]
    y_diff = satellitePosition[1]
    z_diff = satellitePosition[2] - 6370
    return np.sqrt(x_diff**2 + y_diff**2 + z_diff**2)

def quadratic_formula(travel_times, satellitePositions, param, x, y, z):
    A = np.zeros((len(satellitePositions), 4))
    for i, satPos in enumerate(satellitePositions):
        A[i, 0] = 2 * (satPos[0] - x)
        A[i, 1] = 2 * (satPos[1] - y)
        A[i, 2] = 2 * (satPos[2] - z)
        A[i, 3] = 2 * 299792.458 * (0.0001 - travel_times[i])

    b = np.zeros(len(satellitePositions))
    for i, satPos in enumerate(satellitePositions):
        b[i] = (satPos[0] - x)**2 + (satPos[1] - y)**2 + (satPos[2] - z)**2 - (299792.458 * (0.0001 - travel_times[i]))**2

    x_solution = np.linalg.lstsq(A, b, rcond=None)[0]

    return x_solution[param - 1]

def generateOnes(numOfSatellites):
    return np.ones((2**numOfSatellites, numOfSatellites))

def error_magnification(numOfSatellites, theta, phi,infty_norm_t):
    x = 0
    y = 0
    z = 6370

    satellitePositions = np.zeros((numOfSatellites, 3))

    for i in range(numOfSatellites):
        mytheta = theta[i]
        myphi = phi[i]
        satellitePositions[i, :] = getSphericalCoordinates(myphi, mytheta)

    Ri_vector = np.zeros(numOfSatellites)
    travel_times = np.zeros(numOfSatellites)

    for i in range(numOfSatellites):
        Ri_vector[i] = calculateRange(satellitePositions[i])
        travel_times[i] = 0.0001 + Ri_vector[i] / 299792.458

    travel_times_change = generateOnes(numOfSatellites) * 0.0000001

    travel_times_row_vector = travel_times

    conditionNumber = 1

    for row in range(2**numOfSatellites):
        travel_times_my_row = (travel_times_row_vector + travel_times_change[row, :])
        x_bar = quadratic_formula(travel_times_my_row, satellitePositions, 1, x, y, z)
        y_bar = quadratic_formula(travel_times_my_row, satellitePositions, 2, x, y, z)
        z_bar = quadratic_formula(travel_times_my_row, satellitePositions, 3, x, y, z)
        d_bar = quadratic_formula(travel_times_my_row, satellitePositions, 4, x, y, z)

        delta_x = abs(x - x_bar)
        delta_y = abs(y - y_bar)
        delta_z = abs(z - z_bar)

        infty_norm_xyz = max(delta_x, delta_y, delta_z)

        emf = infty_norm_xyz / (infty_norm_t * 299792.458)

        if row == 0:
            conditionNumber = emf
        else:
            if emf > conditionNumber:
                conditionNumber = emf
    print("emf =",emf )
    print("position error found =",infty_norm_xyz)
    return conditionNumber

# Example usage

numOfSatellites = 4
theta = [0,(np.pi/3), ((np.pi)*2)/3, 2* (np.pi)]
phi = [0,(np.pi)/6, (np.pi)/3, (np.pi)/2]

conditionNumber = error_magnification(numOfSatellites, theta, phi,0.00000001)
theConditionNumber = conditionNumber
travel_times_change = generateOnes(numOfSatellites) * 0.00000001
theGeneratingTrow = travel_times_change[np.argmax([error_magnification(numOfSatellites, theta, phi + (np.arange(numOfSatellites) - 0.5) * 0.00000001,0.00000001) for phi in phi])]
positionMinusEMF = abs(theConditionNumber - (theConditionNumber * 299792.458 * theGeneratingTrow).max() / 0.00000001)
print("theConditionNumber =", theConditionNumber)



emf = 2124802.9929628475
position error found = 6369.999120260886
emf = 2124803.299581752
position error found = 6370.000039481238
emf = 2124803.383582912
position error found = 6370.000291310379
emf = 2124803.4501124118
position error found = 6370.000490760803
emf = 2124803.3155750195
position error found = 6370.000087427847
theConditionNumber = 2124802.9929628475


In [23]:
# Problem 3

import numpy as np


# Example usage
numOfSatellites = 4
theta = [0.1,0.105, 0.110, 0.115]
phi = [0.25,0.2625, 0.275, 0.2875]

# with error
print("with error")
conditionNumber = error_magnification(numOfSatellites, theta, phi,0.00000001)
theConditionNumber = conditionNumber

travel_times_change = generateOnes(numOfSatellites) * 0.00000001
theGeneratingTrow = travel_times_change[np.argmax([error_magnification(numOfSatellites, theta, phi + (np.arange(numOfSatellites) - 0.5) * 0.00000001,0.00000001) for phi in phi])]
positionMinusEMF = abs(theConditionNumber - (theConditionNumber * 299792.458 * theGeneratingTrow).max() / 0.00000001)
print("theConditionNumber =", theConditionNumber)


print("\nwithout error")

conditionNumber = error_magnification(numOfSatellites, theta, phi,0)
theConditionNumber = conditionNumber

travel_times_change = generateOnes(numOfSatellites) * 0
theGeneratingTrow = travel_times_change[np.argmax([error_magnification(numOfSatellites, theta, phi + (np.arange(numOfSatellites) - 0.5) * 0,0) for phi in phi])]
positionMinusEMF = abs(theConditionNumber - (theConditionNumber * 299792.458 * theGeneratingTrow).max() / 0)
print("theConditionNumber =", theConditionNumber)


with error
emf = 2125736.9315818413
position error found = 6372.79899780298
emf = 2888266.419341038
position error found = 8658.804892131084
emf = 2048959.8173419288
position error found = 6142.626999841678
emf = 2195229.491320265
position error found = 6581.132450769919
emf = 2046357.126836302
position error found = 6134.824330000727
theConditionNumber = 2125736.9315818413

without error
emf = inf
position error found = 6372.79899780298
emf = inf
position error found = 4220.211099656892
emf = inf
position error found = 6729.058026236874
emf = inf
position error found = 6604.771875126114
emf = inf
position error found = 6479.850196066513
theConditionNumber = inf


<ipython-input-21-86b6f6cce584>:74: RuntimeWarning: divide by zero encountered in double_scalars
  emf = infty_norm_xyz / (infty_norm_t * 299792.458)
<ipython-input-23-a20bbf2c7633>:29: RuntimeWarning: invalid value encountered in multiply
  positionMinusEMF = abs(theConditionNumber - (theConditionNumber * 299792.458 * theGeneratingTrow).max() / 0)
